In [21]:
#测试一下embedding层的作用
from keras.models import Sequential
from keras.layers import Embedding
import numpy as np
model = Sequential()
model.add(Embedding(1000, 64, input_length=10))
# the model will take as input an integer matrix of size (batch, input_length).
# the largest integer (i.e. word index) in the input should be no larger than 999 (vocabulary size).
# now model.output_shape == (None, 10, 64), where None is the batch dimension.

input_array = np.random.randint(1000, size=(32, 10))

model.compile('rmsprop', 'mse')
output_array = model.predict(input_array)
assert output_array.shape == (32, 10, 64) 

input_array每一个元素（如326），经过one-hot编码后都会变成1000维，即input_array经过编码后会变成（32，10，1000）

而Embedding作用是把这一千维数据变成64维

In [4]:
print (input_array)

[[326 348 976 717 202 397  77 331  85 692]
 [176 790 712 234 416   7 669 469 411  82]
 [  2 854 693  97   5 701 569  49 634 701]
 [465   2 487 900 372 375 483 316  28 472]
 [386 805 574 534   9 798 968 151 519 292]
 [479 918 461 591 131 448 192 376 365 290]
 [539 712 591  88 752 371 946 330 490 761]
 [914 715 749 291 351 669 657 221 346 237]
 [629 267  52 922 649 625 378 436 885 492]
 [955 449 822 179 337 590  48 652 491 763]
 [106 187 357 176 251 224 912 589 259 735]
 [ 42 536  94 229 927 187 770 829 163 588]
 [460 947 653 470 367 632 880 801 409 369]
 [871 767 328 511 228 283 142 373 890 617]
 [937 357 753 984 680 300 831 210 622  57]
 [866 512 723  38 914 107 582 319 708  10]
 [  1 676 670 493 903 231 241 475 117 658]
 [144 154 164 156 693 382 437 101 703 881]
 [326 446 947 714 635 150 828 789 373 436]
 [261 469 693  88  36 950 881 194 341 340]
 [476 504 246 698 846  84 387 376 222 971]
 [380 594 409 948 779 340  88  51 710 359]
 [  6 962 446  36 579 166 122 188 819 880]
 [243 605 2

In [7]:
print (output_array.shape)
print (len(output_array))
print (len(output_array[0]))
print (output_array)

(32, 10, 64)
32
10
[[[-0.02658536 -0.01701321 -0.03457935 ..., -0.04842854 -0.03582474
   -0.00322653]
  [ 0.00309172  0.04459726  0.04020951 ...,  0.04615075  0.01565008
   -0.00477158]
  [-0.00445703 -0.01037259 -0.01947093 ..., -0.02401226  0.04409889
   -0.01025251]
  ..., 
  [ 0.01289723  0.02258544 -0.03763982 ...,  0.00990425  0.04037798
   -0.00246155]
  [-0.01281414  0.04575102  0.03102264 ..., -0.02146184  0.04342185
   -0.04540529]
  [-0.00603373  0.04661009 -0.02075096 ...,  0.01052649  0.01878437
   -0.02818539]]

 [[ 0.00186364  0.01617878  0.00865879 ...,  0.01675928  0.02637963
   -0.02079076]
  [-0.01190627  0.01763814  0.04854817 ..., -0.00856728  0.047076
    0.02425525]
  [-0.03893224  0.04383809  0.01461554 ...,  0.00249942 -0.01176268
   -0.04266398]
  ..., 
  [ 0.00776853  0.00077814  0.01685435 ..., -0.02465484 -0.03616484
   -0.03607576]
  [-0.02803652 -0.02433647 -0.00335371 ..., -0.02212117 -0.01351787
    0.04571564]
  [ 0.01029637  0.01400338 -0.00846877 ..

## 下面开始LSTM

In [1]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
# X_train原本是一个60000*28*28的三维向量，将其转换为60000*784的二维向量
X_train = X_train.reshape(60000, 784)
# X_test原本是一个10000*28*28的三维向量，将其转换为10000*784的二维向量
X_test = X_test.reshape(10000, 784)
# 将X_train, X_test的数据格式转为float32
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# 归一化
X_train /= 255
X_test /= 255
# 打印出相关信息
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (60000, 784)
60000 train samples
10000 test samples


In [4]:
batch_size = 128
nb_classes = 10
nb_epoch = 12

In [5]:
# 将类别向量(从0到nb_classes的整数向量)映射为二值类别矩阵，
# 相当于将向量用one-hot重新编码
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [6]:
import tensorflow as tf
import keras

In [113]:
from keras.layers import Merge
from keras.optimizers import RMSprop,adam

left_branch = Sequential()
left_branch.add(Dense(256, input_dim=784))
left_branch.add(Activation('relu'))

right_branch = Sequential()
right_branch.add(Dense(256, input_dim=784))
right_branch.add(Activation('relu'))

merged = Merge([left_branch, right_branch], mode='concat')
#merged = keras.layers.concatenate([left_branch, right_branch])

model = Sequential()
model.add(merged)
model.add(Dropout(0.2))

# # 第二个隐层有512个神经元，激活函数为ReLu，Dropout比例为0.2
# model.add(Dense(512))
# model.add(Activation('relu'))
# model.add(Dropout(0.2))


model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit([X_train,X_train], Y_train,
                    batch_size = batch_size,
                    epochs = nb_epoch,
                    verbose = 1,
                    validation_split=0.2)

D:\anaconda1\lib\site-packages\ipykernel\__main__.py:12: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Train on 48000 samples, validate on 12000 samples
Epoch 1/12
48000/48000 [==============================] - 4s - loss: 0.3021 - acc: 0.9122 - val_loss: 0.1542 - val_acc: 0.9568
Epoch 2/12
48000/48000 [==============================] - 4s - loss: 0.1332 - acc: 0.9609 - val_loss: 0.1123 - val_acc: 0.9672
Epoch 3/12
48000/48000 [==============================] - 4s - loss: 0.0916 - acc: 0.9726 - val_loss: 0.0901 - val_acc: 0.9735
Epoch 4/12
48000/48000 [==============================] - 3s - loss: 0.0704 - acc: 0.9791 - val_loss: 0.0874 - val_acc: 0.9753
Epoch 5/12
48000/48000 [==============================] - 4s - loss: 0.0566 - acc: 0.9824 - val_loss: 0.0821 - val_acc: 0.9768
Epoch 6/12
48000/48000 [==============================] - 3s - loss: 0.0461 - acc: 0.9857 - val_loss: 0.0882 - val_acc: 0.9758
Epoch 7/12
48000/48000 [==============================] - 3s - loss: 0.0389 - acc: 0.9885 - val_loss: 0.0784 - val_acc: 0.9793
Epoch 8/12
48000/48000 [==============================] - 3s 

In [108]:
from keras.layers import Merge
from keras.optimizers import RMSprop,adam


left_branch = Sequential()
left_branch.add(Dense(256, input_dim=784))


right_branch = Sequential()
right_branch.add(Dense(256, input_dim=784))


#merged = Merge([left_branch, right_branch], mode='concat')
merged = keras.layers.concatenate([left_branch.output, right_branch.output])

x = Dense(512, activation='relu')(merged)
dense=Dense(5,activation="softmax",name='aux_output')(x)

from keras.models import Model
merged2=Model(inputs=[left_branch.input,right_branch.input],outputs=[dense])

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit([X_train,X_train], Y_train,
                    batch_size = batch_size,
                    epochs = nb_epoch,
                    verbose = 1,
                    validation_split=0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/12
48000/48000 [==============================] - 4s - loss: 7.8883e-04 - acc: 0.9998 - val_loss: 0.1623 - val_acc: 0.9817
Epoch 2/12
48000/48000 [==============================] - 3s - loss: 6.6159e-04 - acc: 0.9998 - val_loss: 0.1622 - val_acc: 0.9805
Epoch 3/12
48000/48000 [==============================] - 3s - loss: 0.0011 - acc: 0.9998 - val_loss: 0.1605 - val_acc: 0.9826
Epoch 4/12
48000/48000 [==============================] - 3s - loss: 2.6461e-04 - acc: 0.9999 - val_loss: 0.1537 - val_acc: 0.9818
Epoch 5/12
48000/48000 [==============================] - 3s - loss: 8.3031e-04 - acc: 0.9998 - val_loss: 0.1568 - val_acc: 0.9822
Epoch 6/12
48000/48000 [==============================] - 3s - loss: 3.9212e-04 - acc: 0.9999 - val_loss: 0.1574 - val_acc: 0.9822
Epoch 7/12
48000/48000 [==============================] - 3s - loss: 4.1637e-04 - acc: 0.9999 - val_loss: 0.1653 - val_acc: 0.9818
Epoch 8/12
48000/48000 [=============

In [27]:
x = Dense(512, activation='relu')(merged)

In [102]:
dense=Dense(10,activation="softmax",name='aux_output')(x)

In [91]:
from keras.models import Model
merged2=Model(inputs=[left_branch.input,right_branch.input],outputs=[dense])

In [92]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])


In [93]:
history = model.fit([X_train,X_train], Y_train,
                    batch_size = batch_size,
                    epochs = nb_epoch,
                    verbose = 1,
                    validation_split=0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/12
48000/48000 [==============================] - 4s - loss: 0.0012 - acc: 0.9997 - val_loss: 0.1583 - val_acc: 0.9827
Epoch 2/12
48000/48000 [==============================] - 3s - loss: 4.7726e-04 - acc: 0.9998 - val_loss: 0.1488 - val_acc: 0.9821
Epoch 3/12
48000/48000 [==============================] - 3s - loss: 7.7453e-04 - acc: 0.9997 - val_loss: 0.1591 - val_acc: 0.9821
Epoch 4/12
48000/48000 [==============================] - 3s - loss: 3.7441e-04 - acc: 0.9999 - val_loss: 0.1607 - val_acc: 0.9811
Epoch 5/12
48000/48000 [==============================] - 4s - loss: 5.9375e-04 - acc: 0.9998 - val_loss: 0.1660 - val_acc: 0.9818
Epoch 6/12
48000/48000 [==============================] - 3s - loss: 7.9072e-04 - acc: 0.9997 - val_loss: 0.1627 - val_acc: 0.9815
Epoch 7/12
48000/48000 [==============================] - 3s - loss: 4.2985e-04 - acc: 0.9998 - val_loss: 0.1545 - val_acc: 0.9821
Epoch 8/12
48000/48000 [=============

In [114]:
score = model.evaluate([X_test,X_test], Y_test, batch_size=16)
# 输出训练好的模型在测试集上的表现
print('Test score:', score[0])
print('Test accuracy:', score[1])

 9808/10000 [============================>.] - ETA: 0sTest score: 0.0754003866823
Test accuracy: 0.9813


In [94]:
predict=model.predict_classes([X_test,X_test])

 9504/10000 [===========================>..] - ETA: 0s

In [95]:
print (predict.shape)

(10000,)


In [96]:
print (predict)

[7 2 1 ..., 4 5 6]
